<a href="https://colab.research.google.com/github/Halix267/Dog-and-Cat-Classification/blob/master/Dog_and_Cat_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow-2.2.0
import tensorflow as tf
print(tf.__version__)

In [ ]:
#!pip install tensorflow-gpu==2.2.0-rc0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
base_dir = '/content/gdrive/My Drive/My-first_project/My-first_project'
train_dir = os.path.join(base_dir, 'training_set')
test_dir = os.path.join(base_dir, 'test_set')

# Directory with  training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with test cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')

# Directory with test dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [ ]:
#Lets check if the ratio of dogs and cats are correct,it should be 32-32

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rescaling all images by 1./255
train_datagen = ImageDataGenerator(  rescale=1./255,
    )
validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(128,128),  
        shuffle=True,
        color_mode='grayscale',
        batch_size=32,
        class_mode='binary')

# Flow test images in batches of 32 using validation_datagen generator
validattion_generator = validation_datagen.flow_from_directory(
        test_dir,
        target_size=(128,128),
        shuffle=True,
        batch_size=32,
        class_mode='binary',
        color_mode='grayscale')



#X_train,Y_train=train_generator.next()
#X_test,Y_test=test_generator.next()

In [ ]:

#model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization
input_shape=(128,128,1)
model=Sequential([
                  Conv2D(16,(3,3),input_shape=input_shape,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()), 
                  MaxPooling2D((2,2)),
                  Dropout(0.2),
                  Conv2D(32,(3,3),activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()), 
                  MaxPooling2D((2,2)),
                  Dropout(0.2),
                  Conv2D(64,(3,3),activation='relu',kernel_initializer=tf.keras.initializers.he_uniform()), 
                  MaxPooling2D((2,2)),
                  Dropout(0.2),
                  Flatten(),
                  
                  Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.00001)),
                  BatchNormalization(),
                  Dropout(0.2),
                 
                  Dense(1,activation='sigmoid')
                
])

In [ ]:
#Compiling the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy','mae']
)


In [ ]:

#LTraining the model
history =model.fit(
        train_generator,
        epochs=20,
        steps_per_epoch=251,
        callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,mode='min'),
        validation_data = validattion_generator,
        verbose=2        
)